In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from IPython.display import display, Markdown

load_dotenv()

llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0.0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    extra_body={"enable_thinking": False},
)

In [ ]:
negative_example_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""请简要解释 {topic}。
    请不要在解释中包含以下内容：
    - 技术术语或复杂词汇
    - 历史背景或具体年代
    - 与其他相关主题的比较
    你的解释应当简洁、直接，并专注于核心概念。""",
)

response = llm.invoke(negative_example_prompt.format(topic="光合作用")).content
display(Markdown(response))

植物利用阳光，把水和空气中的物质变成养分，并释放出氧气。

In [3]:
exclusion_prompt = PromptTemplate(
    input_variables=["topic", "exclude"],
    template="""请写一小段关于 {topic} 的文字。
    重要提示：不要提及或参考与 {exclude} 有关的任何内容。""",
)

response = llm.invoke(
    exclusion_prompt.format(topic="锻炼的好处", exclude="减重或身体形象")
).content
display(Markdown(response))

锻炼有助于增强心肺功能，提升整体身体素质，同时还能促进大脑释放内啡肽，改善情绪、缓解压力。长期坚持适度的运动，可以增强免疫力，提高睡眠质量，并提升日常生活的活力与专注力。

In [11]:
constraint_prompt = PromptTemplate(
    input_variables=["topic", "style", "excluded_words"],
    template="""请用 {style} 风格描述 {topic}。
    输出要求：
    1. 不要使用以下词语：{excluded_words}
    2. 描述控制在 100 个词以内
    3. 不要使用类比或比喻
    4. 内容应完全基于事实""",
)

response = llm.invoke(
    constraint_prompt.format(
        topic="人工智能", style="技术性", excluded_words="机器人、人类、科幻"
    )
).content
display(Markdown(response))

人工智能是一种基于算法的计算系统，通过数据训练实现特定任务的自动化处理。其核心技术包括机器学习、深度学习和自然语言处理等，依赖数学模型与统计方法进行模式识别与决策优化。人工智能系统通过输入数据生成输出结果，并在迭代过程中不断调整参数以提升性能。

In [12]:
def evaluate_output(output, constraints):
    """评估输出是否满足给定的约束条件。"""
    results = {}
    for constraint, check_func in constraints.items():
        results[constraint] = check_func(output)
    return results

In [13]:
import re

constraints = {
    "word_count": lambda x: len(x.split()) <= 100,
    "no_excluded_words": lambda x: all(word not in x for word in ["机器人", "人类", "科幻"]),
    "no_analogies": lambda x: not re.search(r"\b(as|like)\b", x, re.IGNORECASE)
}

In [14]:
evaluation_results = evaluate_output(response, constraints)
print("评估结果:", evaluation_results)

评估结果: {'word_count': True, 'no_excluded_words': True, 'no_analogies': True}


In [15]:
if not all(evaluation_results.values()):
    refined_prompt = constraint_prompt.format(
        topic="artificial intelligence",
        style="technical and concise",  # 添加“concise”以控制字数
        excluded_words="机器人, 人类, 科幻, 像, 比如"  # 增加“像”和“比如”以避免使用类比
    )
    refined_response = llm.invoke(refined_prompt).content
    print("\n优化后的输出：\n", refined_response)

    # 再次评估优化后的输出
    refined_evaluation = evaluate_output(refined_response, constraints)
    print("\n优化后的评估结果:", refined_evaluation)
